In [1]:
import xgboost as xgb
import numpy as np
import json
import pandas as pd# Used for Regression Modelling
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

In [28]:
data = pd.read_csv('./data/merged_mldata2020v2.csv',delimiter = ',')
with open(f'score_predictor.pkl', 'rb') as f:
    model = pickle.load(f)

outputs = []
games = data.drop(columns =['vscore', 'hscore','winner','ou',"sWinner","fspread","fscore","vospread","vcspread"])
games = pd.DataFrame(games)

weekfilter = games['week'] == 1

games = games[weekfilter]
#print(games)
columnheads = ['temp','wind_mph','vdflg','hdflg','divgame','nsite','hospread','ouopen','hcspread','ouclose','vTOTAL.DVOA','vTOTAL.RNK','vOFF.RNK','vOFF.DVOA','vDEF.RNK','vDEF.DVOA','vST.RNK','vST.DVOA','hTOTAL.DVOA','hTOTAL.RNK','hOFF.RNK','hOFF.DVOA','hDEF.RNK','hDEF.DVOA','hST.RNK','hST.DVOA','vtsw','vtsl','vtst','htsw','htsl','htst','vtw','vtl','vtt','vts','htw','htl','htt','hts']

games = np.hsplit(games,[3])

tw = games[0]
pdata = games[1]



input_variables = pd.DataFrame(pdata, columns = columnheads,dtype=float)

#prediction = model.predict(input_variables)

input_variables["prediction"] = model.predict(input_variables)

#print(prediction)
#print(input_variables)

output_info = input_variables[["hcspread","ouclose","prediction"]]
output_info = tw.join(output_info)

output_info["vcspread"] = np.where(output_info["hcspread"] >= 0,"-" + output_info["hcspread"].astype(str),output_info["hcspread"].astype(str).str.replace("-","+"))
output_info["hcspread"] = np.where(output_info["hcspread"] >= 0,"+" + output_info["hcspread"].astype(str),output_info["hcspread"])


output_info["prediction"] = np.where(output_info["prediction"] < 0, output_info["vteam"] +" "+ output_info["vcspread"].astype(str) , output_info["hteam"] +" "+ output_info["hcspread"].astype(str))
result = output_info.to_json(orient="table")

parsed = json.loads(result)

#json.dumps(parsed, indent=10)  
print(parsed["data"][0])
print(parsed["data"][4])


#for i in games.iterrows():
    #prediction = {'vteam':i['vteam'],'hteam':i['hteam']}
    #input_variables = x.drop(columns=['week','vteam','hteam'])
    #prediction['outcome'] = model.predict(input_variables)[0]
    #print(i['vteam'])


#print(x)
#print(teams)

{'index': 0, 'week': 1, 'vteam': 'HOU', 'hteam': 'KC', 'hcspread': -9.5, 'ouclose': 53.5, 'prediction': 'HOU +9.5', 'vcspread': '+9.5'}
{'index': 4, 'week': 1, 'vteam': 'LV', 'hteam': 'CAR', 'hcspread': '+3.0', 'ouclose': 48.0, 'prediction': 'LV -3.0', 'vcspread': '-3.0'}
